In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

print("🗺️ CUSTOMER JOURNEY MAPPING ANALYSIS")
print("=" * 60)

class CustomerJourneyMapper:
    """
    Advanced customer journey mapping and path analysis
    Maps touchpoints, identifies bottlenecks, and optimizes conversion paths
    """
    
    def __init__(self, transaction_data):
        self.transaction_data = transaction_data
        self.journey_data = {}
        self.touchpoint_sequences = []
        self.journey_segments = {}
        
    def simulate_customer_touchpoints(self):
        """Simulate realistic customer touchpoint data across the journey"""
        
        print("🔍 Simulating customer touchpoint data...")
        
        # Define touchpoint types and their properties
        touchpoints = {
            'awareness': {
                'social_media_ad': {'stage': 'awareness', 'digital': True, 'cost': 2.50, 'conversion_rate': 0.008},
                'display_ad': {'stage': 'awareness', 'digital': True, 'cost': 3.20, 'conversion_rate': 0.005},
                'content_marketing': {'stage': 'awareness', 'digital': True, 'cost': 0.80, 'conversion_rate': 0.012},
                'word_of_mouth': {'stage': 'awareness', 'digital': False, 'cost': 0.00, 'conversion_rate': 0.025},
                'pr_coverage': {'stage': 'awareness', 'digital': False, 'cost': 0.50, 'conversion_rate': 0.008}
            },
            'consideration': {
                'google_search': {'stage': 'consideration', 'digital': True, 'cost': 0.00, 'conversion_rate': 0.035},
                'paid_search': {'stage': 'consideration', 'digital': True, 'cost': 2.80, 'conversion_rate': 0.055},
                'comparison_site': {'stage': 'consideration', 'digital': True, 'cost': 1.20, 'conversion_rate': 0.025},
                'review_reading': {'stage': 'consideration', 'digital': True, 'cost': 0.00, 'conversion_rate': 0.018},
                'product_demo': {'stage': 'consideration', 'digital': False, 'cost': 15.00, 'conversion_rate': 0.180}
            },
            'evaluation': {
                'website_visit': {'stage': 'evaluation', 'digital': True, 'cost': 0.00, 'conversion_rate': 0.065},
                'product_page': {'stage': 'evaluation', 'digital': True, 'cost': 0.00, 'conversion_rate': 0.045},
                'cart_addition': {'stage': 'evaluation', 'digital': True, 'cost': 0.00, 'conversion_rate': 0.320},
                'live_chat': {'stage': 'evaluation', 'digital': True, 'cost': 2.50, 'conversion_rate': 0.280},
                'email_inquiry': {'stage': 'evaluation', 'digital': True, 'cost': 0.10, 'conversion_rate': 0.190}
            },
            'purchase': {
                'checkout_start': {'stage': 'purchase', 'digital': True, 'cost': 0.00, 'conversion_rate': 0.680},
                'payment_page': {'stage': 'purchase', 'digital': True, 'cost': 0.00, 'conversion_rate': 0.820},
                'order_confirmation': {'stage': 'purchase', 'digital': True, 'cost': 0.00, 'conversion_rate': 1.000}
            },
            'post_purchase': {
                'order_tracking': {'stage': 'post_purchase', 'digital': True, 'cost': 0.00, 'conversion_rate': 0.750},
                'delivery': {'stage': 'post_purchase', 'digital': False, 'cost': 8.50, 'conversion_rate': 0.950},
                'customer_service': {'stage': 'post_purchase', 'digital': True, 'cost': 5.00, 'conversion_rate': 0.880},
                'review_request': {'stage': 'post_purchase', 'digital': True, 'cost': 0.05, 'conversion_rate': 0.220},
                'satisfaction_survey': {'stage': 'post_purchase', 'digital': True, 'cost': 0.02, 'conversion_rate': 0.150}
            }
        }
        
        # Flatten touchpoints
        all_touchpoints = {}
        for stage, tps in touchpoints.items():
            all_touchpoints.update(tps)
        
        # Simulate customer journeys
        customer_journeys = []
        journey_id = 1
        
        for _, transaction in self.transaction_data.iterrows():
            customer_id = transaction['customer_id']
            purchase_date = transaction['date']
            purchase_value = transaction['total_amount']
            device = transaction.get('device', 'Desktop')
            
            # Simulate journey path
            journey_path = self._generate_journey_path(all_touchpoints, purchase_value)
            
            # Create touchpoint records
            for i, (touchpoint_name, days_before) in enumerate(journey_path):
                touchpoint_date = purchase_date - timedelta(days=days_before)
                touchpoint_info = all_touchpoints[touchpoint_name]
                
                journey_record = {
                    'journey_id': journey_id,
                    'customer_id': customer_id,
                    'touchpoint_sequence': i + 1,
                    'touchpoint_name': touchpoint_name,
                    'touchpoint_stage': touchpoint_info['stage'],
                    'touchpoint_date': touchpoint_date,
                    'is_digital': touchpoint_info['digital'],
                    'touchpoint_cost': touchpoint_info['cost'],
                    'conversion_rate': touchpoint_info['conversion_rate'],
                    'device': device,
                    'purchase_value': purchase_value,
                    'days_to_purchase': days_before
                }
                
                customer_journeys.append(journey_record)
            
            journey_id += 1
        
        self.journey_data = pd.DataFrame(customer_journeys)
        print(f"✅ Generated {len(customer_journeys)} touchpoint interactions across {journey_id-1} customer journeys")
        
        return self.journey_data
    
    def _generate_journey_path(self, touchpoints, purchase_value):
        """Generate a realistic customer journey path"""
        
        # Define journey complexity based on purchase value
        if purchase_value < 50:
            # Simple journey
            path_length = np.random.randint(3, 6)
        elif purchase_value < 200:
            # Medium journey
            path_length = np.random.randint(5, 9)
        else:
            # Complex journey
            path_length = np.random.randint(7, 12)
        
        # Define stage progression probabilities
        stage_progression = {
            'awareness': ['consideration', 'evaluation'],
            'consideration': ['evaluation', 'purchase', 'awareness'],
            'evaluation': ['purchase', 'consideration'],
            'purchase': ['post_purchase'],
            'post_purchase': []
        }
        
        # Start with awareness stage
        current_stage = 'awareness'
        journey_path = []
        days_back = np.random.randint(1, 45)  # Start 1-45 days before purchase
        
        for step in range(path_length):
            # Select touchpoint from current stage
            stage_touchpoints = [tp for tp, info in touchpoints.items() 
                               if info['stage'] == current_stage]
            
            if stage_touchpoints:
                selected_touchpoint = np.random.choice(stage_touchpoints)
                journey_path.append((selected_touchpoint, days_back))
                
                # Progress to next stage or stay
                if current_stage in stage_progression and stage_progression[current_stage]:
                    if np.random.random() > 0.3:  # 70% chance to progress
                        current_stage = np.random.choice(stage_progression[current_stage])
                
                # Reduce days back (move closer to purchase)
                days_back = max(0, days_back - np.random.randint(1, 5))
        
        # Ensure purchase touchpoints are included
        purchase_touchpoints = ['checkout_start', 'payment_page', 'order_confirmation']
        for i, tp in enumerate(purchase_touchpoints):
            journey_path.append((tp, i))
        
        return journey_path
    
    def analyze_journey_paths(self):
        """Analyze common journey paths and sequences"""
        
        print("📊 Analyzing customer journey paths...")
        
        if self.journey_data.empty:
            self.simulate_customer_touchpoints()
        
        # Path analysis
        path_analysis = {}
        
        # Common touchpoint sequences
        sequences = []
        for journey_id in self.journey_data['journey_id'].unique():
            journey = self.journey_data[self.journey_data['journey_id'] == journey_id]
            journey = journey.sort_values('touchpoint_sequence')
            sequence = ' → '.join(journey['touchpoint_name'].tolist())
            sequences.append(sequence)
        
        sequence_counts = pd.Series(sequences).value_counts()
        path_analysis['common_sequences'] = sequence_counts.head(10)
        
        # Stage transitions
        transitions = []
        for journey_id in self.journey_data['journey_id'].unique():
            journey = self.journey_data[self.journey_data['journey_id'] == journey_id]
            journey = journey.sort_values('touchpoint_sequence')
            stages = journey['touchpoint_stage'].tolist()
            
            for i in range(len(stages) - 1):
                transitions.append(f"{stages[i]} → {stages[i+1]}")
        
        transition_counts = pd.Series(transitions).value_counts()
        path_analysis['stage_transitions'] = transition_counts
        
        # Journey length analysis
        journey_lengths = self.journey_data.groupby('journey_id').size()
        path_analysis['avg_journey_length'] = journey_lengths.mean()
        path_analysis['journey_length_dist'] = journey_lengths.value_counts().sort_index()
        
        # Time to purchase analysis
        time_analysis = self.journey_data.groupby('journey_id')['days_to_purchase'].max()
        path_analysis['avg_time_to_purchase'] = time_analysis.mean()
        
        self.touchpoint_sequences = path_analysis
        
        print(f"✅ Analyzed {len(self.journey_data['journey_id'].unique())} unique customer journeys")
        print(f"📈 Average journey length: {path_analysis['avg_journey_length']:.1f} touchpoints")
        print(f"⏱️ Average time to purchase: {path_analysis['avg_time_to_purchase']:.1f} days")
        
        return path_analysis
    
    def identify_conversion_bottlenecks(self):
        """Identify bottlenecks and drop-off points in the customer journey"""
        
        print("🚧 Identifying conversion bottlenecks...")
        
        if self.journey_data.empty:
            self.simulate_customer_touchpoints()
        
        # Calculate conversion rates by touchpoint
        touchpoint_performance = self.journey_data.groupby('touchpoint_name').agg({
            'conversion_rate': 'mean',
            'touchpoint_cost': 'mean',
            'journey_id': 'nunique',
            'purchase_value': 'mean'
        }).round(3)
        
        touchpoint_performance.columns = ['avg_conversion_rate', 'avg_cost', 'frequency', 'avg_purchase_value']
        
        # Calculate efficiency metrics
        touchpoint_performance['cost_per_conversion'] = (
            touchpoint_performance['avg_cost'] / touchpoint_performance['avg_conversion_rate']
        ).replace([np.inf, -np.inf], 0)
        
        touchpoint_performance['roi'] = (
            (touchpoint_performance['avg_purchase_value'] * touchpoint_performance['avg_conversion_rate'] - 
             touchpoint_performance['avg_cost']) / touchpoint_performance['avg_cost']
        ).replace([np.inf, -np.inf], 0)
        
        # Identify bottlenecks (low conversion, high cost)
        bottlenecks = touchpoint_performance[
            (touchpoint_performance['avg_conversion_rate'] < touchpoint_performance['avg_conversion_rate'].median()) &
            (touchpoint_performance['avg_cost'] > touchpoint_performance['avg_cost'].median())
        ].sort_values('cost_per_conversion', ascending=False)
        
        # Identify high performers
        high_performers = touchpoint_performance[
            (touchpoint_performance['avg_conversion_rate'] > touchpoint_performance['avg_conversion_rate'].quantile(0.75)) &
            (touchpoint_performance['roi'] > 0)
        ].sort_values('roi', ascending=False)
        
        bottleneck_analysis = {
            'touchpoint_performance': touchpoint_performance,
            'bottlenecks': bottlenecks,
            'high_performers': high_performers
        }
        
        print(f"⚠️ Identified {len(bottlenecks)} potential bottlenecks")
        print(f"🌟 Identified {len(high_performers)} high-performing touchpoints")
        
        return bottleneck_analysis
    
    def segment_customer_journeys(self, n_segments=4):
        """Segment customers based on their journey patterns"""
        
        print(f"👥 Segmenting customers into {n_segments} journey types...")
        
        if self.journey_data.empty:
            self.simulate_customer_touchpoints()
        
        # Create journey features for clustering
        journey_features = []
        
        for journey_id in self.journey_data['journey_id'].unique():
            journey = self.journey_data[self.journey_data['journey_id'] == journey_id]
            
            features = {
                'journey_id': journey_id,
                'customer_id': journey['customer_id'].iloc[0],
                'journey_length': len(journey),
                'total_cost': journey['touchpoint_cost'].sum(),
                'digital_touchpoints': journey['is_digital'].sum(),
                'time_to_purchase': journey['days_to_purchase'].max(),
                'purchase_value': journey['purchase_value'].iloc[0],
                
                # Stage distribution
                'awareness_touchpoints': (journey['touchpoint_stage'] == 'awareness').sum(),
                'consideration_touchpoints': (journey['touchpoint_stage'] == 'consideration').sum(),
                'evaluation_touchpoints': (journey['touchpoint_stage'] == 'evaluation').sum(),
                'purchase_touchpoints': (journey['touchpoint_stage'] == 'purchase').sum(),
                'post_purchase_touchpoints': (journey['touchpoint_stage'] == 'post_purchase').sum(),
            }
            
            journey_features.append(features)
        
        journey_df = pd.DataFrame(journey_features)
        
        # Prepare features for clustering
        feature_cols = [col for col in journey_df.columns if col not in ['journey_id', 'customer_id']]
        X = journey_df[feature_cols]
        
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Perform clustering
        kmeans = KMeans(n_clusters=n_segments, random_state=42)
        journey_df['segment'] = kmeans.fit_predict(X_scaled)
        
        # Analyze segments
        segment_profiles = journey_df.groupby('segment').agg({
            'journey_length': 'mean',
            'total_cost': 'mean',
            'digital_touchpoints': 'mean',
            'time_to_purchase': 'mean',
            'purchase_value': 'mean',
            'customer_id': 'count'
        }).round(2)
        
        segment_profiles.columns = ['avg_journey_length', 'avg_total_cost', 'avg_digital_touchpoints', 
                                  'avg_time_to_purchase', 'avg_purchase_value', 'customer_count']
        
        # Add segment labels
        segment_labels = {
            0: 'Quick Converters',
            1: 'Researchers', 
            2: 'Bargain Hunters',
            3: 'Premium Buyers'
        }
        
        segment_profiles['segment_label'] = [segment_labels.get(i, f'Segment {i}') 
                                           for i in segment_profiles.index]
        
        self.journey_segments = {
            'journey_features': journey_df,
            'segment_profiles': segment_profiles,
            'scaler': scaler,
            'model': kmeans
        }
        
        print("✅ Customer journey segmentation completed")
        for idx, row in segment_profiles.iterrows():
            print(f"  {row['segment_label']}: {row['customer_count']} customers, "
                  f"avg journey length: {row['avg_journey_length']:.1f}")
        
        return self.journey_segments
    
    def create_journey_visualizations(self):
        """Create comprehensive visualizations of customer journey analysis"""
        
        print("📊 Creating customer journey visualizations...")
        
        if self.journey_data.empty:
            self.simulate_customer_touchpoints()
        
        if not self.touchpoint_sequences:
            self.analyze_journey_paths()
        
        # 1. Journey Flow Sankey Diagram
        fig_sankey = self._create_sankey_diagram()
        
        # 2. Touchpoint Performance Matrix
        fig_performance = self._create_touchpoint_performance_matrix()
        
        # 3. Journey Timeline Heatmap
        fig_timeline = self._create_journey_timeline_heatmap()
        
        # 4. Conversion Funnel
        fig_funnel = self._create_conversion_funnel()
        
        visualizations = {
            'sankey_diagram': fig_sankey,
            'performance_matrix': fig_performance,
            'timeline_heatmap': fig_timeline,
            'conversion_funnel': fig_funnel
        }
        
        print("✅ Journey visualizations created successfully")
        
        return visualizations
    
    def _create_sankey_diagram(self):
        """Create Sankey diagram showing journey flow"""
        
        # Prepare data for Sankey
        stage_transitions = []
        for journey_id in self.journey_data['journey_id'].unique():
            journey = self.journey_data[self.journey_data['journey_id'] == journey_id]
            journey = journey.sort_values('touchpoint_sequence')
            stages = journey['touchpoint_stage'].tolist()
            
            for i in range(len(stages) - 1):
                stage_transitions.append((stages[i], stages[i+1]))
        
        transition_counts = pd.Series(stage_transitions).value_counts()
        
        # Create nodes and links
        all_stages = list(set([t[0] for t in stage_transitions] + [t[1] for t in stage_transitions]))
        stage_to_idx = {stage: idx for idx, stage in enumerate(all_stages)}
        
        source = [stage_to_idx[t[0]] for t in transition_counts.index]
        target = [stage_to_idx[t[1]] for t in transition_counts.index]
        value = transition_counts.values
        
        fig = go.Figure(data=[go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=all_stages,
                color="blue"
            ),
            link=dict(
                source=source,
                target=target,
                value=value
            )
        )])
        
        fig.update_layout(
            title_text="Customer Journey Flow - Stage Transitions",
            font_size=10,
            height=600
        )
        
        return fig
    
    def _create_touchpoint_performance_matrix(self):
        """Create performance matrix of touchpoints"""
        
        performance = self.journey_data.groupby(['touchpoint_name', 'touchpoint_stage']).agg({
            'conversion_rate': 'mean',
            'touchpoint_cost': 'mean',
            'journey_id': 'nunique'
        }).reset_index()
        
        fig = px.scatter(
            performance,
            x='conversion_rate',
            y='touchpoint_cost',
            size='journey_id',
            color='touchpoint_stage',
            hover_name='touchpoint_name',
            title='Touchpoint Performance Matrix',
            labels={
                'conversion_rate': 'Conversion Rate',
                'touchpoint_cost': 'Cost per Touchpoint',
                'journey_id': 'Frequency'
            }
        )
        
        fig.update_layout(height=600)
        return fig
    
    def _create_journey_timeline_heatmap(self):
        """Create heatmap showing touchpoint timing patterns"""
        
        # Create pivot table for heatmap
        timeline_data = self.journey_data.pivot_table(
            values='journey_id',
            index='touchpoint_name',
            columns='days_to_purchase',
            aggfunc='count',
            fill_value=0
        )
        
        fig = px.imshow(
            timeline_data.values,
            x=timeline_data.columns,
            y=timeline_data.index,
            aspect='auto',
            title='Customer Journey Timeline Heatmap',
            labels={'x': 'Days Before Purchase', 'y': 'Touchpoint', 'color': 'Frequency'}
        )
        
        fig.update_layout(height=800)
        return fig
    
    def _create_conversion_funnel(self):
        """Create conversion funnel by stage"""
        
        stage_order = ['awareness', 'consideration', 'evaluation', 'purchase', 'post_purchase']
        stage_counts = self.journey_data['touchpoint_stage'].value_counts()
        
        # Order by funnel stage
        ordered_counts = [stage_counts.get(stage, 0) for stage in stage_order]
        
        fig = go.Figure(go.Funnel(
            y=stage_order,
            x=ordered_counts,
            textinfo="value+percent initial"
        ))
        
        fig.update_layout(
            title="Customer Journey Conversion Funnel",
            height=600
        )
        
        return fig
    
    def generate_journey_insights(self):
        """Generate actionable insights from journey analysis"""
        
        print("💡 Generating customer journey insights...")
        
        # Ensure all analyses are completed
        if self.journey_data.empty:
            self.simulate_customer_touchpoints()
        
        if not self.touchpoint_sequences:
            path_analysis = self.analyze_journey_paths()
        else:
            path_analysis = self.touchpoint_sequences
        
        bottleneck_analysis = self.identify_conversion_bottlenecks()
        
        if not self.journey_segments:
            segment_analysis = self.segment_customer_journeys()
        else:
            segment_analysis = self.journey_segments
        
        # Generate insights
        insights = {
            'journey_overview': {
                'total_journeys': len(self.journey_data['journey_id'].unique()),
                'avg_journey_length': path_analysis['avg_journey_length'],
                'avg_time_to_purchase': path_analysis['avg_time_to_purchase'],
                'total_touchpoints': len(self.journey_data)
            },
            
            'top_performing_touchpoints': bottleneck_analysis['high_performers'].head(5),
            'major_bottlenecks': bottleneck_analysis['bottlenecks'].head(5),
            'common_journey_paths': path_analysis['common_sequences'].head(5),
            'stage_transition_patterns': path_analysis['stage_transitions'].head(10),
            'customer_segments': segment_analysis['segment_profiles'],
            
            'recommendations': self._generate_recommendations(bottleneck_analysis, segment_analysis)
        }
        
        print("✅ Journey insights generated successfully")
        
        return insights
    
    def _generate_recommendations(self, bottleneck_analysis, segment_analysis):
        """Generate actionable recommendations"""
        
        recommendations = []
        
        # Bottleneck recommendations
        if not bottleneck_analysis['bottlenecks'].empty:
            worst_bottleneck = bottleneck_analysis['bottlenecks'].iloc[0]
            recommendations.append({
                'category': 'Bottleneck Optimization',
                'priority': 'High',
                'recommendation': f"Optimize '{worst_bottleneck.name}' touchpoint - high cost ({worst_bottleneck['avg_cost']:.2f}) with low conversion rate ({worst_bottleneck['avg_conversion_rate']:.3f})",
                'expected_impact': 'Reduce customer acquisition cost by 15-25%'
            })
        
        # High performer recommendations
        if not bottleneck_analysis['high_performers'].empty:
            best_performer = bottleneck_analysis['high_performers'].iloc[0]
            recommendations.append({
                'category': 'Scale High Performers',
                'priority': 'Medium',
                'recommendation': f"Increase investment in '{best_performer.name}' - shows high ROI ({best_performer['roi']:.2f}) and conversion rate ({best_performer['avg_conversion_rate']:.3f})",
                'expected_impact': 'Increase overall conversion rate by 10-20%'
            })
        
        # Segment-specific recommendations
        segment_profiles = segment_analysis['segment_profiles']
        largest_segment = segment_profiles.loc[segment_profiles['customer_count'].idxmax()]
        
        recommendations.append({
            'category': 'Segment Optimization',
            'priority': 'Medium',
            'recommendation': f"Focus on optimizing journey for '{largest_segment['segment_label']}' segment ({largest_segment['customer_count']} customers) with avg journey length of {largest_segment['avg_journey_length']:.1f} touchpoints",
            'expected_impact': 'Improve experience for largest customer group'
        })
        
        # Journey length optimization
        avg_length = segment_analysis['journey_features']['journey_length'].mean()
        if avg_length > 8:
            recommendations.append({
                'category': 'Journey Simplification',
                'priority': 'High',
                'recommendation': f"Simplify customer journey - current average of {avg_length:.1f} touchpoints may indicate friction",
                'expected_impact': 'Reduce time to purchase and improve conversion rates'
            })
        
        return recommendations

# Example usage and demonstration
def demonstrate_journey_mapping():
    """Demonstrate the CustomerJourneyMapper with sample data"""
    
    print("\n🚀 CUSTOMER JOURNEY MAPPING DEMONSTRATION")
    print("=" * 50)
    
    # Create sample transaction data
    np.random.seed(42)
    n_customers = 1000
    
    sample_data = []
    for i in range(n_customers):
        customer_id = f"CUST_{i+1:04d}"
        purchase_date = datetime.now() - timedelta(days=np.random.randint(1, 90))
        total_amount = np.random.exponential(100) + 20  # Exponential distribution for realistic amounts
        device = np.random.choice(['Desktop', 'Mobile', 'Tablet'], p=[0.5, 0.4, 0.1])
        
        sample_data.append({
            'customer_id': customer_id,
            'date': purchase_date,
            'total_amount': total_amount,
            'device': device
        })
    
    transaction_df = pd.DataFrame(sample_data)
    
    # Initialize journey mapper
    journey_mapper = CustomerJourneyMapper(transaction_df)
    
    # Perform complete analysis
    print("\n1. Simulating customer touchpoints...")
    journey_data = journey_mapper.simulate_customer_touchpoints()
    
    print("\n2. Analyzing journey paths...")
    path_analysis = journey_mapper.analyze_journey_paths()
    
    print("\n3. Identifying bottlenecks...")
    bottleneck_analysis = journey_mapper.identify_conversion_bottlenecks()
    
    print("\n4. Segmenting customers...")
    segment_analysis = journey_mapper.segment_customer_journeys()
    
    print("\n5. Generating insights...")
    insights = journey_mapper.generate_journey_insights()
    
    print("\n6. Creating visualizations...")
    visualizations = journey_mapper.create_journey_visualizations()
    
    # Display key insights
    print("\n📈 KEY INSIGHTS SUMMARY")
    print("-" * 30)
    print(f"Total Customer Journeys Analyzed: {insights['journey_overview']['total_journeys']}")
    print(f"Average Journey Length: {insights['journey_overview']['avg_journey_length']:.1f} touchpoints")
    print(f"Average Time to Purchase: {insights['journey_overview']['avg_time_to_purchase']:.1f} days")
    
    print("\n🌟 TOP PERFORMING TOUCHPOINTS:")
    for touchpoint, metrics in insights['top_performing_touchpoints'].head(3).iterrows():
        print(f"  • {touchpoint}: {metrics['avg_conversion_rate']:.1%} conversion, ROI: {metrics['roi']:.2f}")
    
    print("\n⚠️ MAJOR BOTTLENECKS:")
    for touchpoint, metrics in insights['major_bottlenecks'].head(3).iterrows():
        print(f"  • {touchpoint}: ${metrics['cost_per_conversion']:.2f} cost per conversion")
    
    print("\n👥 CUSTOMER SEGMENTS:")
    for idx, segment in insights['customer_segments'].iterrows():
        print(f"  • {segment['segment_label']}: {segment['customer_count']} customers, "
              f"${segment['avg_purchase_value']:.0f} avg purchase")
    
    print("\n💡 TOP RECOMMENDATIONS:")
    for rec in insights['recommendations'][:3]:
        print(f"  • [{rec['priority']}] {rec['recommendation']}")
    
    return journey_mapper, insights, visualizations

if __name__ == "__main__":
    # Run the demonstration
    mapper, insights, viz = demonstrate_journey_mapping()
    
    print("\n✅ Customer Journey Mapping Analysis Complete!")
    print("📊 Visualizations and detailed insights are ready for review.")

🗺️ CUSTOMER JOURNEY MAPPING ANALYSIS

🚀 CUSTOMER JOURNEY MAPPING DEMONSTRATION

1. Simulating customer touchpoints...
🔍 Simulating customer touchpoint data...
✅ Generated 9333 touchpoint interactions across 1000 customer journeys

2. Analyzing journey paths...
📊 Analyzing customer journey paths...
✅ Analyzed 1000 unique customer journeys
📈 Average journey length: 9.3 touchpoints
⏱️ Average time to purchase: 22.6 days

3. Identifying bottlenecks...
🚧 Identifying conversion bottlenecks...
⚠️ Identified 6 potential bottlenecks
🌟 Identified 2 high-performing touchpoints

4. Segmenting customers...
👥 Segmenting customers into 4 journey types...
✅ Customer journey segmentation completed
  Quick Converters: 204 customers, avg journey length: 10.1
  Researchers: 350 customers, avg journey length: 7.4
  Bargain Hunters: 326 customers, avg journey length: 9.8
  Premium Buyers: 120 customers, avg journey length: 12.3

5. Generating insights...
💡 Generating customer journey insights...
🚧 Identifyi